In [15]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import numpy as np
import skimage
from skimage.io import imread, imshow
from skimage.transform import resize

In [22]:
def readImgs(path, length):
    imgs = []
    i = 0
    for filename in os.listdir(path): 
        i += 1
        if i == length:
            break
        if filename.endswith('.jpg'):
            img = imread(os.path.join(path,filename))
            resized = resize(img, (256, 256), anti_aliasing=True)
            imgs.append(resized)
    return imgs

In [25]:
train_df_path = 'deepfake_database/deepfake_database/train:test/df'
train_real_path = 'deepfake_database/deepfake_database/train:test/real'
deepfakes_train = readImgs(train_df_path, 21)
real_train = readImgs(train_real_path, 21)

In [26]:
print(len(deepfakes_train))
print(len(real_train))

20
20


In [33]:
y_train = [0]*20 + [1]*20
X_train = deepfakes_train + real_train

In [34]:
y_train = np.array(y_train)
X_train = np.array(X_train)

In [35]:
X_train.shape

(40, 256, 256, 3)

# MesoNet Model Development

In [2]:
import tensorflow as tf
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Reshape, Concatenate, LeakyReLU

Using TensorFlow backend.


In [3]:
def init_model(img_width): 
    x = Input(shape = (img_width, img_width, 3))
    x1 = Conv2D(8, (3, 3), padding='same', activation = 'relu')(x)
    x1 = BatchNormalization()(x1)
    x1 = MaxPooling2D(pool_size=(2, 2), padding='same')(x1)
        
    x2 = Conv2D(8, (5, 5), padding='same', activation = 'relu')(x1)
    x2 = BatchNormalization()(x2)
    x2 = MaxPooling2D(pool_size=(2, 2), padding='same')(x2)
        
    x3 = Conv2D(16, (5, 5), padding='same', activation = 'relu')(x2)
    x3 = BatchNormalization()(x3)
    x3 = MaxPooling2D(pool_size=(2, 2), padding='same')(x3)
        
    x4 = Conv2D(16, (5, 5), padding='same', activation = 'relu')(x3)
    x4 = BatchNormalization()(x4)
    x4 = MaxPooling2D(pool_size=(4, 4), padding='same')(x4)
        
    y = Flatten()(x4)
    y = Dropout(0.5)(y)
    y = Dense(16)(y)
    y = LeakyReLU(alpha=0.1)(y)
    y = Dropout(0.5)(y)
    y = Dense(1, activation = 'sigmoid')(y)
    return Model(inputs = x, outputs = y)

In [4]:
Meso4 = init_model(256)
opt = Adam(lr = 0.001)
Meso4.compile(optimizer = opt, loss = 'binary_crossentropy')

In [48]:
Meso4.fit(X_train, y_train, batch_size = 50, validation_split = 0.2, epochs = 5)

Train on 400 samples, validate on 100 samples
Epoch 1/5
400/400 [==============================] - 30s 76ms/step - loss: 2.0292 - val_loss: 0.7274
Epoch 2/5
400/400 [==============================] - 21s 52ms/step - loss: 1.1109 - val_loss: 0.6889
Epoch 3/5
400/400 [==============================] - 20s 49ms/step - loss: 0.8715 - val_loss: 0.7026
Epoch 4/5
400/400 [==============================] - 23s 57ms/step - loss: 0.7349 - val_loss: 0.7127
Epoch 5/5
400/400 [==============================] - 24s 59ms/step - loss: 0.6808 - val_loss: 0.7215


In [ ]:
Meso4.save('Meso4_limited_dataset.h5')

In [8]:
from keras.preprocessing.image import ImageDataGenerator

In [9]:
dataGenerator = ImageDataGenerator(rescale=1./255)
generator = dataGenerator.flow_from_directory(
        'extracted_imgs',
        shuffle=False,
        target_size=(256, 256),
        batch_size=5,
        class_mode='binary',
        subset='training')

# Predict 
# y: Deepfake, Deepfake, Deepfake, real, real 
X_test, y_test = generator.next()
image_names = generator.filenames

Found 5 images belonging to 2 classes.


In [11]:
def convertToActual(probs):
    return np.round(probs)

def computeAccuracy(pred, real):
    cnt = 0
    for i in range(len(pred)):
        if pred[i] == real[i]:
            cnt += 1
    return cnt/len(pred)*100

# Predictions from training on a limited portion of the dataset

In [38]:
print("The order of the files are (df is DeepFake):")
for name in image_names:
        print(name)
Meso4.load_weights('Meso4_limited_dataset.h5')
prob_real_meso = Meso4.predict(X_test)

actual_pred_Meso = convertToActual(prob_real_meso)
print('Predicted probability of the image being real Meso:', prob_real_meso,'\nPredicted class :', actual_pred_Meso)
print('Deepfake detection accuracy is:', computeAccuracy(actual_pred_Meso, y_test), '%')

The order of the files are (df is DeepFake):
df/elon_fake_1.png
df/elon_fake_2.png
df/obama_fake.png
real/elon_real.png
real/obama_real.png
Predicted probability of the image being real Meso: [[0.4955473 ]
 [0.51936394]
 [0.55302715]
 [0.5075346 ]
 [0.49840945]] 
Predicted class : [[0.]
 [1.]
 [1.]
 [1.]
 [0.]]
Deepfake detection accuracy is: 40.0 %


# Predictions from the pretrained weights

In [31]:
Meso4.load_weights('weights/Meso4_DF')

In [32]:
print("The order of the files are (df is DeepFake):")
for name in image_names:
        print(name)
prob_real_meso = Meso4.predict(X_test)

actual_pred_Meso = convertToActual(prob_real_meso)
print('Predicted probability of the image being real Meso:', prob_real_meso,'\nPredicted class :', actual_pred_Meso)
print('Deepfake detection accuracy is:', computeAccuracy(actual_pred_Meso, y_test), '%')

The order of the files are (df is DeepFake):
df/elon_fake_1.png
df/elon_fake_2.png
df/obama_fake.png
real/elon_real.png
real/obama_real.png
Predicted probability of the image being real Meso: [[0.7889354]
 [0.7816247]
 [0.9934752]
 [0.8929086]
 [0.695085 ]] 
Predicted class : [[1.]
 [1.]
 [1.]
 [1.]
 [1.]]
Deepfake detection accuracy is: 40.0 %


## Predictions on the original data set for a sanity check 

In [38]:
prob_real_meso_1 = Meso4.predict(X_train)

actual_pred_Meso_1 = convertToActual(prob_real_meso_1)
print('Predicted probability of the image being real Meso:', prob_real_meso_1,'\nPredicted class :', actual_pred_Meso_1)
print('Deepfake detection accuracy is:', computeAccuracy(actual_pred_Meso_1, y_train), '%')

Predicted probability of the image being real Meso: [[4.5588583e-01]
 [4.5874006e-01]
 [5.1707745e-02]
 [1.3030857e-02]
 [2.5985420e-02]
 [2.8252310e-01]
 [4.6801388e-02]
 [5.0092816e-01]
 [4.8294336e-02]
 [1.2809092e-01]
 [6.7514610e-01]
 [6.3818693e-04]
 [3.1593686e-01]
 [2.6893723e-01]
 [1.1758542e-01]
 [1.3589859e-03]
 [4.5870692e-01]
 [4.8491436e-01]
 [6.4844346e-01]
 [5.6062639e-03]
 [7.2420782e-01]
 [7.1361661e-01]
 [7.3447573e-01]
 [9.5594561e-01]
 [8.9558512e-01]
 [9.6427691e-01]
 [7.7566338e-01]
 [8.1778729e-01]
 [5.9420520e-01]
 [4.6097699e-01]
 [5.5239248e-01]
 [7.5305945e-01]
 [9.6896005e-01]
 [3.0411106e-01]
 [3.7789840e-01]
 [2.9373610e-01]
 [9.0920424e-01]
 [9.8764241e-01]
 [9.7903252e-01]
 [8.1950206e-01]] 
Predicted class : [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]]
Deepfak